In [ ]:
# imports
import numpy as np
import pandas as pd
import pyodbc
import warnings
from efficient_apriori import apriori
from itertools import permutations
warnings.filterwarnings('ignore')

# 2. Conexão com SQL Server 
server = '*********'
database = '**********'
username = '*******'
password = '*******'

conn_str = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'


with pyodbc.connect(conn_str) as conn:
        cursor = conn.cursor()

sql = """
SELECT 
D2_DOC, //NOTA FISCAL
D2_COD, //CÓDIGO DO PRODUTO
CASE WHEN A1_CODSEG = '' THEN '000000' else A1_CODSEG END AS A1_CODSEG // CÓDIGO DE SEGMENTO
FROM SD2010 SD2
INNER JOIN SB1010 SB1 ON B1_COD = D2_COD AND B1_FILIAL = '01' AND SB1.D_E_L_E_T_ = ''
INNER JOIN SA1010 SA1 ON A1_COD = D2_CLIENTE AND A1_LOJA = D2_LOJA AND A1_FILIAL = '01'
WHERE D2_FILIAL = '01IN04' AND B1_TIPO = 'PA' AND D2_EMISSAO >= '20200101' AND SD2.D_E_L_E_T_ = '' AND A1_SATIV5 <> '000801' AND B1_MSBLQL = '2'
""" 
df = pd.read_sql(sql, conn)

In [2]:
#Agrupando os produtos por segmento e nota
transacooes_segmento = df.groupby(["A1_CODSEG", "D2_DOC"])["D2_COD"].apply(list).reset_index()


In [3]:
transacooes_segmento

,A1_CODSEG,D2_DOC,D2_COD
0,000000,000380190,[02100042001240 ]
1,000000,000380194,[02100031001105 ]
2,000000,000380203,[02100021001085 ]
3,000000,000380215,"[02100031001105 , 02100021001321 ]"
4,000000,000380238,"[02100042001255 , 02100031001105 , 02100021001..."
...,...,...,...
488140,000140,001076322,"[02100042001240 , 02100031001105 , 02100021001..."
488141,000140,001076330,"[02100021001061 , 02100021001086 , 02100021001..."
488142,000140,001076337,[02100042001240 ]
488143,000140,001076340,[02100042001240 ]


In [4]:
#Organizar em dicionário por segmento
segmentos = {}
for seg in df['A1_CODSEG'].unique():
    transacoes = transacooes_segmento[transacooes_segmento['A1_CODSEG'] == seg]['D2_COD'].tolist()
    segmentos[seg] = [tuple(x) for x in transacoes]

In [5]:
resultados_por_segmento = {}

for segmento, transacoes in segmentos.items():
    #Aplicar Apriori com parâmetros específico para cada segmento
    itemsets, rules = apriori(transacoes, min_support = 0.02, min_confidence = 0.2)

    #Order regras por lift
    rules.sort(key=lambda x:x.lift, reverse = True)

    #Armazenar resultados
    resultados_por_segmento[segmento] =  {
    'itemsets': itemsets,
    'rules': rules,
    'transacoes': len(transacoes)
    }
    

In [23]:
analise_comparativa = [
    {
        'Segmento': segmento,
        'N_Transacoes': dados['transacoes'],
        'Top_5_Regra': [
            {
                'Antecedente': rule.lhs,
                'Consequente': rule.rhs,
                'Suporte': rule.support,
                'Confianca': rule.confidence,
                'Lift': rule.lift
            } for rule in dados['rules'][:5]
        ]
    }
    for segmento, dados in resultados_por_segmento.items() 
    if dados['rules']  # Filtra segmentos com regras
]

In [ ]:
df = pd.DataFrame([
    {
        'Segmento': item['Segmento'],
        'N_transacoes': item['N_Transacoes'],
        'Antecedente': regra['Antecedente'],
        'Consequente': regra['Consequente'],
        'Suporte': regra['Suporte'],
        'Confianca': regra['Confianca'],
        'Lift': regra['Lift']
    }
    for item in analise_comparativa
    for regra in item['Top_5_Regra']
])
df.sort_values(by = ['Segmento', 'Lift'], ascending = [True, False])
df.head(50)
df.to_excel('Resultado.xlsx', index=False)

,Segmento,N_transacoes,Antecedente,Consequente,Suporte,Confianca,Lift
0,000085,9517,"(02100031001101 , 02100031001221 )","(02100021001025 , 02100031001104 )",0.021961,0.643077,15.037256
1,000085,9517,"(02100021001025 , 02100031001104 )","(02100031001101 , 02100031001221 )",0.021961,0.513514,15.037256
2,000085,9517,"(02100031001101 , 02100031001104 )","(02100021001025 , 02100031001221 )",0.021961,0.441860,14.965075
3,000085,9517,"(02100021001025 , 02100031001221 )","(02100031001101 , 02100031001104 )",0.021961,0.743772,14.965075
4,000085,9517,"(02100021001025 , 02100031001104 , 02100031001...","(02100031001101 ,)",0.021961,0.863636,13.883830
